# h3) CC Summaries
Produces summary, agrregated results from the daily power and energy modelling.




## Libraries and Setup

In [1]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math
import statsmodels.tsa.stattools as statools

## Parameters

In [2]:
#Papermill parameters
input_file='ngonye_daily_WC1_1_base_pe'#'ngonye_daily'

In [10]:
input_data='./input_data/'
output_data='./output_data/CC/'

In [11]:
model_name='Base Case'#MW153#MW144 'MW162'#'Fixed Tailwater'#'Base Case' Headpond50cm EWRAllCs
dryrun=False #Dont write output files

In [12]:
models=pd.read_csv(input_data + "models.csv").set_index('ModelName')
model=models.loc[model_name]

out_prefix=model['OutputPrefix']#'base'
if out_prefix!='base':
    out_prefix="scenarios/" + out_prefix
    
plant_capacity=model['Capacity']#180
canal_capacity=plant_capacity/180*1100#1100

model

OutputPrefix                       base
Description                   Base Case
EWRCatSet              Recommendation 1
HeadpondLift                          0
Capacity                            180
FixedTailwater                        0
Mean                             832484
P95                              718504
P90                              756610
P75                              792216
P50                              838165
P25                              876248
P10                              921915
P5                               958253
StDev                           81971.2
CoefVar                       0.0984658
Max                         1.01897e+06
Min                              493939
CapFactor                      0.527958
CalYearMean                      832587
CalYearStDev                    78237.2
CalYearCoefVar                0.0939688
AutoCorrel                          NaN
Power_max                       193.908
CapFactor_Power_max             0.49015


## Load Daily Data

Load the daily time series which includes the exceedance values used for calculating daily EWRs.

In [13]:
daily=pd.read_csv(output_data + input_file + '.csv') #ngonye_daily_WC1_1_base_pe
daily=daily.set_index(pd.to_datetime(daily['Date'],format="%Y-%m-%d"))
daily=daily.drop('Date',axis=1)
daily.head(5)

,Year,Day,Month,WaterYear,WaterMonth,MonthId,WaterDay,WaterWeek,CC_Factor,SampleDate,...,EffGen2,EffGen3,EffGen4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec
Date,,,,,,,,,,,,,,,,,,,,,
2017-10-01,2017,1,10,2017,1,2017.1,1,1,0.3375,1933-10-01,...,NaN,NaN,NaN,16.790064,NaN,NaN,NaN,16.790064,402.961538,0.094047
2017-10-02,2017,2,10,2017,1,2017.1,2,1,0.3375,1933-10-02,...,NaN,NaN,NaN,16.790064,NaN,NaN,NaN,16.790064,402.961538,0.094047
2017-10-03,2017,3,10,2017,1,2017.1,3,1,0.3375,1933-10-03,...,NaN,NaN,NaN,16.790064,NaN,NaN,NaN,16.790064,402.961538,0.094047
2017-10-04,2017,4,10,2017,1,2017.1,4,1,0.3375,1933-10-04,...,NaN,NaN,NaN,16.216244,NaN,NaN,NaN,16.216244,389.189859,0.092271
2017-10-05,2017,5,10,2017,1,2017.1,5,1,0.3375,1933-10-05,...,NaN,NaN,NaN,16.216244,NaN,NaN,NaN,16.216244,389.189859,0.092271


## Monthly


In [15]:

monthly=daily.groupby("MonthId").mean().drop(['Day','EWRBandNo','EWRRefExceedance','Energy'],axis=1)

monthly=monthly.join(daily.groupby("MonthId").agg(    
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),   
   Turbines_max=('Turbines', 'max'),    
   Turbines_min=('Turbines', 'min'),   
   Energy=('Energy', 'sum'),
))
monthly['Days']=monthly.apply(lambda x: pd.Period(year=x['Year'],month=x['Month'],freq='D').days_in_month,axis=1)
monthly['CapFactor']=monthly['Energy']/(plant_capacity*24*monthly['Days'])

monthly.tail(12)

,Year,Month,WaterYear,WaterMonth,WaterDay,WaterWeek,CC_Factor,SampleFlow,Impact_2070,Flow,...,PowerTurbine4,Power,PerfMwPerCumec,Power_max,Power_min,Turbines_max,Turbines_min,Energy,Days,CapFactor
MonthId,,,,,,,,,,,,,,,,,,,,,
2052.10,2052.0,10.0,2052.0,1.0,16.0,2.741935,0.7750,305.412179,-0.16,267.541068,...,NaN,34.327174,0.127868,42.383823,29.548406,1,1,25539.417260,31,0.190707
2052.11,2052.0,11.0,2052.0,2.0,46.5,7.066667,0.7750,356.388894,-0.20,301.148616,...,NaN,42.610331,0.139440,60.304864,28.556457,2,1,30679.438284,30,0.236724
2052.12,2052.0,12.0,2052.0,3.0,77.0,11.419355,0.7750,553.985963,-0.17,480.998312,...,NaN,76.154629,0.157652,95.045876,59.056827,2,2,56659.043820,31,0.423081
2053.01,2053.0,1.0,2052.0,4.0,108.0,15.870968,0.7875,790.184416,-0.18,678.175775,...,NaN,110.206561,0.162373,137.973050,91.567529,3,2,81993.681745,31,0.612259
2053.02,2053.0,2.0,2052.0,5.0,137.5,20.071429,0.7875,1755.235893,-0.16,1534.076170,...,40.575389,157.172342,0.127544,190.684241,115.696476,4,3,105619.813553,28,0.873180
2053.03,2053.0,3.0,2052.0,6.0,167.0,24.258065,0.7875,6893.057191,-0.41,4667.461351,...,24.672525,98.690102,0.021763,128.321585,0.000000,4,0,73425.435795,31,0.548278
2053.04,2053.0,4.0,2052.0,7.0,197.5,28.666667,0.7875,7422.717087,-0.29,5727.554072,...,8.980505,35.922019,0.007506,108.583744,0.000000,4,0,25863.853941,30,0.199567
2053.05,2053.0,5.0,2052.0,8.0,228.0,33.000000,0.7875,3738.769494,-0.24,3032.142060,...,34.548092,138.192368,0.049206,163.412260,107.297588,4,4,102815.122128,31,0.767735
2053.06,2053.0,6.0,2052.0,9.0,258.5,37.333333,0.7875,1862.872723,-0.24,1510.789778,...,43.501242,172.349069,0.117643,187.379979,147.011765,4,3,124091.329704,30,0.957495


## Yearly

In [16]:
yearly=daily.groupby("WaterYear").agg(
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
)
yearly['CapFactor']=yearly['Energy']/(plant_capacity*24*365)


In [17]:
yearly['Prop4Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==4,1,0))/365)
yearly['Prop3Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==3,1,0))/365)
yearly['Prop2Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==2,1,0))/365)
yearly['Prop1Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==1,1,0))/365)
yearly['Prop0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0))/365)
yearly['Days0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0)))


## Calendar Months

In [18]:
calmonthly=pd.DataFrame({'WaterMonth': [1,2,3,4,5,6,7,8,9,10,11,12],'MonthName': ['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep'], 'Month':[10,11,12,1,2,3,4,5,6,7,8,9]})
calmonthly=calmonthly.set_index('WaterMonth')
calmonthly=calmonthly.join(monthly.groupby(["WaterMonth"]).agg(    
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_mean=('EWRProportion', 'mean'),
   EWRProportion_median=('EWRProportion', 'median'),
   EWRProportion_min=('EWRProportion', 'min'),
    
    EWRTotal_max=('EWRTotal', 'max'),
   EWRTotal_mean=('EWRTotal', 'mean'),
   EWRTotal_median=('EWRTotal', 'median'),
   EWRTotal_min=('EWRTotal', 'min'),   
    
   FlowCanal_max=('FlowCanal', 'max'),
   FlowCanal_mean=('FlowCanal', 'mean'),
   FlowCanal_median=('FlowCanal', 'median'),
   FlowCanal_min=('FlowCanal', 'min'),     
    
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_mean=('HeadTurbine1', 'mean'),
   HeadTurbine1_median=('HeadTurbine1', 'median'),
   HeadTurbine1_min=('HeadTurbine1', 'min'), 

   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_mean=('LevelTailwater', 'mean'),    
   LevelTailwater_median=('LevelTailwater', 'median'),    
   LevelTailwater_min=('LevelTailwater', 'min'),  
   LevelHeadpond_max=('LevelHeadpond', 'max'),
   LevelHeadpond_mean=('LevelHeadpond', 'mean'),    
   LevelHeadpond_median=('LevelHeadpond', 'median'),    
   LevelHeadpond_min=('LevelHeadpond', 'min'),  
   Turbines_mean=('Turbines', 'mean'),      
   Power_max=('Power', 'max'),
   Power_mean=('Power', 'mean'),
   Power_median=('Power', 'median'),
   Power_min=('Power', 'min'),    
   Energy_max=('Energy', 'max'),
   Energy_mean=('Energy', 'mean'),
   Energy_median=('Energy', 'median'),    
   Energy_min=('Energy', 'min'),
))
calmonthly['Energy_P95']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.05)
calmonthly['Energy_P90']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.10)
calmonthly['Energy_P75']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.25)
calmonthly['Energy_P25']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.75)
calmonthly['Energy_P10']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.90)
calmonthly['Energy_P5']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.95)

calmonthly['Energy_stdev']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').std()
calmonthly['Energy_coefvar']=calmonthly['Energy_stdev']/calmonthly['Energy_mean']

calmonthly

,MonthName,Month,EWRProportion_max,EWRProportion_mean,EWRProportion_median,EWRProportion_min,EWRTotal_max,EWRTotal_mean,EWRTotal_median,EWRTotal_min,...,Energy_median,Energy_min,Energy_P95,Energy_P90,Energy_P75,Energy_P25,Energy_P10,Energy_P5,Energy_stdev,Energy_coefvar
WaterMonth,,,,,,,,,,,,,,,,,,,,,
1,Oct,10,0.496064,0.413267,0.413948,0.325567,120.0,93.655018,97.000000,52.0,...,20166.272357,15447.754616,16795.057008,17417.140777,18339.172688,24130.916010,25867.124384,29386.451978,4147.278641,0.194338
2,Nov,11,0.418602,0.349914,0.358188,0.257784,114.0,89.472222,95.166667,50.0,...,25054.524838,18984.929541,20234.055485,20778.164976,22530.852525,29854.078242,32686.184076,35631.114958,5604.737089,0.211547
3,Dec,12,0.293462,0.249871,0.254411,0.179589,126.0,100.472222,104.000000,59.0,...,44499.097273,37384.633949,39437.883881,40337.532837,42325.514914,54456.155534,62993.836520,75105.500308,11646.520007,0.233986
4,Jan,1,0.233585,0.185773,0.191414,0.108941,150.0,118.277778,126.129032,73.0,...,73655.215566,52377.311949,57071.703802,63363.111936,67302.109975,85630.951997,105580.140566,126620.932987,19497.008329,0.244644
5,Feb,2,0.179753,0.131039,0.141110,0.026742,197.0,139.322660,137.000000,85.0,...,95764.137014,7712.678171,65160.467742,76025.441545,89057.593320,107971.302532,114734.614832,116996.306206,20848.454953,0.220605
6,Mar,3,0.196364,0.098700,0.097701,0.054868,315.0,195.833333,192.935484,107.0,...,109773.933627,29633.185228,61611.374791,74610.206408,99742.904282,119637.320635,128508.326486,130575.433613,24167.682898,0.231584
7,Apr,4,0.172183,0.084941,0.081772,0.064789,364.0,233.527778,243.000000,125.0,...,102175.664328,25863.853941,77788.998221,79490.695169,88373.589627,116915.479886,122338.028500,127369.758536,20288.743846,0.201782
8,May,5,0.246596,0.141301,0.141539,0.078109,340.0,266.194444,290.516129,147.0,...,125183.397670,71792.316707,96914.410162,105957.555686,114423.516325,130055.648096,133427.272755,135195.798364,14767.229007,0.122369
9,Jun,6,0.335802,0.219471,0.218060,0.127919,293.0,213.944444,210.000000,119.0,...,106762.058518,37481.399954,67590.404880,80754.118725,84938.966101,119212.303098,123807.502971,124110.006222,20976.709332,0.207747


## Save Files

In [20]:
outp=output_data + input_file

if dryrun==False:
    #daily.to_csv(outp+'_pe_daily.csv')
    monthly.to_csv(outp+'_monthly.csv')
    yearly.to_csv(outp+'_yearly.csv')
    #edc.to_csv(outp+'_pe_percentiles.csv')
    #monthly_day.to_csv(outp+'_pe_monthly_day.csv')
    #daily_slim.to_csv(outp+'_pe_daily_slim.csv')
    #weekly.to_csv(outp+'_pe_weekly.csv')
    #weekly_slim.to_csv(outp+'_pe_weekly_slim.csv')
    calmonthly.to_csv(outp+'_calmonthly.csv')
    #power_exceed.to_csv(outp+'_pe_power_exceed.csv')
    #fdc.to_csv(outp+'_pe_fdc.csv')
    #autocorrel.to_csv(outp+'_pe_autocorrel.csv')
    #models.to_csv(input_data + "models.csv")